In [2]:
import requests
from requests.auth import HTTPBasicAuth, HTTPDigestAuth
from selenium import webdriver
from icalendar import Calendar, Event, vCalAddress, vText
import pytz
from datetime import datetime
import os
from pathlib import Path
import re
import configparser

In [9]:
config = configparser.ConfigParser()
config.read('config.ini')

'22104597'

In [4]:
DRIVER_PATH = 'msedgedriver.exe'
driver = webdriver.Edge(executable_path=DRIVER_PATH)
driver.implicitly_wait(10)
driver.get('https://cas.ut-capitole.fr/cas/login?service=https%3A%2F%2Fade-production.ut-capitole.fr%2Fdirect%2Fmyplanning.jsp')

In [5]:
user = driver.find_element_by_id("username")
user.send_keys(config.get('default', 'username'))
pas = driver.find_element_by_id("password")
pas.send_keys(config.get('default', 'password'))
login = driver.find_element_by_id("connexion").click()

In [35]:
legend = driver.find_elements_by_class_name("slot")
planning = driver.find_element_by_id("Planning")
planning_div = planning.find_elements_by_xpath("./*")
planning_data = planning.find_elements_by_xpath("//div[@aria-owns='Planning']")

In [57]:
hours = {}
for l in legend:
    heure = l.text[:-1] if len(l.text) > 5 else l.text
    top = l.get_attribute("style").split("; ")[-1]
    top_nb = int(re.findall(r'\d+', top)[0])+8
    hours[top_nb] = heure

In [85]:
from datetime import date, timedelta

cal = Calendar()
today = date.today()
current = today - timedelta(days=today.weekday())
current_top = 0

for div, data in zip(planning_div, planning_data):
    top = div.get_attribute("style").split("; ")[-1]
    top_nb = int(re.findall(r'\d+', top)[0])
    list_data = [x.strip() for x in list(filter(lambda x: x.replace(" ", "") != "", data.get_attribute("aria-label").split('null')))]

    if current_top > top_nb:
        current = current + timedelta(days=1)
    current_top = top_nb

    print(current)
    print(list_data)
    year = current.year
    month = current.month
    day = current.day

    hour_class = hours[top_nb].split("h")
    print(hour_class)
    event = Event()
    event.add('summary', list_data[0])
    event.add('dtstart', datetime(year, month, day, int(hour_class[0]), int(hour_class[1])))
    event.add('dtend', datetime(year, month, day, int(hour_class[0])+3, int(hour_class[1])))

    if len(list_data) in [4, 5]:
        event['location'] = vText(list_data[1])
    else:
        event['location'] = vText('Inconnu')
    # Adding events to calendar
    cal.add_component(event)

2022-03-14
['CM Gouvernance des SI', 'MH003', 'PELFRESNE RENE', 'IMMBA1CM01', 'IMMBB1CM01']
['09', '30']
2022-03-14
['TD Intégration de données', 'MH003', 'RAVAT FRANCK', 'IMMBA1TD01', 'IMMBB1TD01']
['14', '00']
2022-03-14
['Projet', 'MH003', 'IMMBA1CM01', 'IMMBB1CM01']
['15', '00']
2022-03-15
['CM Intégration de données', 'ME402', 'SONG JIEFU', 'IMMBA1CM01', 'IMMBB1CM01']
['09', '30']
2022-03-15
['CM Modélisation du SI', 'MH003', 'ZURFLUH GILLES', 'IMMBA1CM01', 'IMMBB1CM01']
['14', '00']
2022-03-16
['Projet', 'IMMBA1CM01', 'IMMBB1CM01']
['09', '30']
2022-03-16
['CM Modélisation du SI', 'MH003', 'ZURFLUH GILLES', 'IMMBA1CM01', 'IMMBB1CM01']
['14', '00']
2022-03-17
['Projet', 'ME407', 'IMMBA1CM01', 'IMMBB1CM01']
['09', '30']
2022-03-17
['TD Intégration de données', 'ME407', 'SONG JIEFU', 'IMMBA1TD01', 'IMMBB1TD01']
['14', '00']
2022-03-18
['CM Gouvernance des SI', 'MF103', 'PELFRESNE RENE', 'IMMBA1CM01', 'IMMBB1CM01']
['09', '30']
2022-03-18
['CM Gouvernance des SI', 'MF105', 'PELFRESNE

In [86]:
f = open(os.path.join("./", 'example.ics'), 'wb')
f.write(cal.to_ical())
f.close()